# Strategy for KNN

for sure i will need to randomly undersample and SMOTE oversample the training set (TRAIN ONLY) to keep balance

Use correlation between features of the dataset

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import time
import random
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline as SkPipeline
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler


import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# load the dataset and fix values

In [2]:
# Use the GPU
if torch.backends.mps.is_available():
    print("MPS device is available.")
    device = torch.device("mps")
elif torch.cuda.is_available():
    print("CUDA device is available.")
    device = torch.device("cuda")
else:
    print("No GPU acceleration available.")
    device = torch.device("cpu")

# Fix the seed to have deterministic behaviour
def fix_random(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True  # slower

SEED = 1337
fix_random(SEED)

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

DATASET_PATH = "dataset_train/dataset.csv"
dataset = pd.read_csv(DATASET_PATH, delimiter=",")

print(f"Shape of the dataset: {dataset.shape}")
duplicates = dataset[dataset.duplicated()]
print(f"Number of duplicates in the dataset: {duplicates.shape[0]}")

MPS device is available.
Shape of the dataset: (148301, 145)
Number of duplicates in the dataset: 0


## split the dataset

In [3]:
X = dataset.drop(columns=["grade"])
y = dataset["grade"].map({"A": 6, "B": 5, "C": 4, "D": 3, "E": 2, "F": 1, "G": 0})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [4]:
class NumericExtractor(BaseEstimator, TransformerMixin):
    """Extracts integers from strings using regex"""

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        for col in X.columns:
            X[col] = X[col].astype(str).str.extract(r"(\d+)").astype(float)
        return X

class CyclicalDateEncoder(BaseEstimator, TransformerMixin):
    """Converts mm-yyyy to year + sine/cosine month encoding."""

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        for col in X.columns:
            # errors="coerce" turns unparseable data/NaNs into NaT
            date_series = pd.to_datetime(X[col], format="%b-%Y", errors="coerce")
            # If date is NaT, these become NaN, which we handle in the pipeline later
            angle = 2 * np.pi * date_series.dt.month / 12

            X[f"{col}_year"] = date_series.dt.year
            X[f"{col}_month_sin"] = np.sin(angle)
            X[f"{col}_month_cos"] = np.cos(angle)
            
            X.drop(columns=[col], inplace=True)
        return X
    
class BinaryModeEncoder(BaseEstimator, TransformerMixin):
    """"Encodes 0 if value is mode, 1 if not"""
    def __init__(self):
        self.modes_ = {}

    def fit(self, X, y=None):
        # Calculate mode for each column and store it
        for col in X.columns:
            self.modes_[col] = X[col].mode()[0]
        return self

    def transform(self, X):
        X_copy = X.copy()
        for col, mode in self.modes_.items():
            # Apply: 1 if NOT the mode (least frequent), 0 if mode
            X_copy[col] = (X_copy[col] != mode).astype(int)
        return X_copy
    
class HighMissingDropper(BaseEstimator, TransformerMixin):
    """Drops columns with high missing percentage. Fits only on training data."""
    
    def __init__(self, threshold=20):
        self.threshold = threshold
        self.cols_to_drop_ = []

    def fit(self, X, y=None):
        # Calculate missing percentages only on training data
        missing_percentages = X.isna().mean() * 100
        self.cols_to_drop_ = missing_percentages[missing_percentages > self.threshold].index.tolist()
        return self

    def transform(self, X):
        X = X.copy()
        return X.drop(columns=self.cols_to_drop_)

In [5]:
redundant_cols = ['loan_title', "borrower_address_zip", "borrower_address_state"] 
binary_cols = ["loan_payment_plan_flag", "listing_initial_status", "application_type_label",
               "hardship_flag_indicator", "disbursement_method_type", "debt_settlement_flag_indicator"]
one_hot_encoding_cols = ["borrower_housing_ownership_status", "borrower_income_verification_status",
                       "loan_status_current_code", "loan_purpose_category"]
extract_fields = ["loan_contract_term_months", "borrower_profile_employment_length"]
date_fields = ["loan_issue_date", "credit_history_earliest_line", "last_payment_date", "last_credit_pull_date"]

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold, cross_val_score

numeric_pipe = SkPipeline([
    ('extract', NumericExtractor()),
    ('impute', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_pipe = SkPipeline([
    ('impute', SimpleImputer(strategy='constant', fill_value='missing')), # Prevent OHE crash
    ('ohe', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
])

date_pipe = SkPipeline([
    ('cyclical', CyclicalDateEncoder()),
    ('impute', SimpleImputer(strategy='median')), # Imputing sin/cos is mathematically valid
    ('scaler', StandardScaler())
])

binary_pipe = SkPipeline([
    ('binary_enc', BinaryModeEncoder()), 
    ('impute', SimpleImputer(strategy='most_frequent')) # Safety net
])

# remainder columns are numerical
remainder_pipe = SkPipeline([
    ('impute', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num_pipe', numeric_pipe, extract_fields),
        ('cat_pipe', categorical_pipe, one_hot_encoding_cols),
        ('date_pipe', date_pipe, date_fields),
        ('bin_pipe', binary_pipe, binary_cols),
        ('drop_redundant', 'drop', redundant_cols)
    ], 
    remainder=remainder_pipe
)

full_pipeline = ImbPipeline([
    ('dropper', HighMissingDropper(threshold=20)),
    ('prep', preprocessor),
    ('sampler', 'passthrough'),
    ('lda', LDA()),
    ('knn', KNeighborsClassifier(weights='distance'))
])

param_grid = {
    'knn__n_neighbors': [3, 5, 7],
    'lda': ['passthrough', LDA()],
    'sampler': ['passthrough', RandomUnderSampler(random_state=SEED), SMOTE(random_state=SEED)]
}

scoring = {
    'acc': 'accuracy',
    'balanced_acc': 'balanced_accuracy',
    'f1_weighted': 'f1_weighted'
}

grid_search = GridSearchCV(
    estimator=full_pipeline,
    param_grid=param_grid,
    cv=3,
    scoring=scoring,
    refit='balanced_acc',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 18 candidates, totalling 54 fits


/Users/geko/unibo/data_analytics/project/.venv/lib/python3.13/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)
/Users/geko/unibo/data_analytics/project/.venv/lib/python3.13/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)
/Users/geko/unibo/data_analytics/project/.venv/lib/python3.13/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)
/Users/geko/unibo/data_analytics/project/.venv/lib/python3.13/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [2] during

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...'distance'))])
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'knn__n_neighbors': [3, 5, ...], 'lda': ['passthrough', LinearDiscriminantAnalysis()], 'sampler': ['passthrough', RandomUnderSa...om_state=1337), ...]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.","{'acc': 'accuracy', 'balanced_acc': 'balanced_accuracy', 'f1_weighted': 'f1_weighted'}"
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",'balanced_acc'
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",3
,"verbos

In [7]:
# Best parameters found
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best CV score: {grid_search.best_score_:.4f}")

print("--"*20)
results = grid_search.cv_results_
best_index = grid_search.best_index_

# Now these keys will exist because you used the dictionary scoring
print(f"Best Model Results (Refit on F1_Weighted):")
print(f"Mean Accuracy: {results['mean_test_acc'][best_index]:.4f}")
print(f"Mean Balanced Acc: {results['mean_test_balanced_acc'][best_index]:.4f}")
print(f"Mean F1 Weighted: {results['mean_test_f1_weighted'][best_index]:.4f}")

print("--"*20)

# Predict using the best model found
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.4f}")
bacc = balanced_accuracy_score(y_test, y_pred)
print(f"Balanced Accuracy: {bacc:.4f}")
f1 = f1_score(y_test, y_pred, average="weighted")
print(f"F1 score: {f1:.4f}")

Best parameters: {'knn__n_neighbors': 7, 'lda': LinearDiscriminantAnalysis(), 'sampler': 'passthrough'}
Best CV score: 0.7531
----------------------------------------
Best Model Results (Refit on F1_Weighted):
Mean Accuracy: 0.8209
Mean Balanced Acc: 0.7531
Mean F1 Weighted: 0.8198
----------------------------------------
Accuracy: 0.8311
Balanced Accuracy: 0.7666
F1 score: 0.8301


In [ ]:
import pickle

# --- 1. Split the Best Estimator ---
full_best_pipeline = grid_search.best_estimator_

# Slicing [:-1] creates a new Pipeline containing ONLY the 'dropper' and 'prep' steps
# It retains all the fitted logic (modes, dropped columns, etc.)
preprocessing_part = full_best_pipeline[:-1] 

# Access the 'rf' step directly to get just the classifier
classifier_part = full_best_pipeline.named_steps['knn']

# --- 2. Save them as separate pickles ---
with open("knn_preprocessor.save", "wb") as f:
    pickle.dump(preprocessing_part, f)

with open("knn_model.save", "wb") as f:
    pickle.dump(classifier_part, f)

print("Saved 'knn_preprocessor.save' and 'knn_model.save' separately.")

Saved 'rf_preprocessor.save' and 'rf_model.save' separately.


In [9]:
print(results)

{'mean_fit_time': array([1.87635128, 1.92945798, 6.27470016, 2.89691265, 1.72984219,
       9.95099123, 1.48834658, 1.33568668, 5.68546867, 2.8749822 ,
       1.67397269, 9.03988036, 1.42234302, 1.21188402, 5.41789118,
       2.81093502, 1.63829589, 6.48764936]), 'std_fit_time': array([0.03915789, 0.13201724, 0.19516032, 0.03506183, 0.05749568,
       0.30590685, 0.14737703, 0.1469968 , 0.24806766, 0.09792389,
       0.03099675, 0.10006959, 0.14680781, 0.11251738, 0.22525275,
       0.15693998, 0.10039523, 0.39571821]), 'mean_score_time': array([17.04064536,  4.94570009, 30.57460952,  1.70684695,  1.12035163,
        2.62706304, 17.30538805,  5.063363  , 28.77925992,  2.35033385,
        1.23324005,  2.34682989, 16.48794794,  4.96332177, 21.66788046,
        2.05066331,  1.04080804,  1.93514172]), 'std_score_time': array([0.23520127, 0.11015464, 0.57324001, 0.15087868, 0.30393923,
       0.48689708, 0.43797172, 0.04762458, 0.47354484, 0.16425654,
       0.05817192, 0.56623492, 0.473060

In [ ]:
df_results = pd.DataFrame(results)

# Filter for the metrics we care about
cols = [
    'param_knn__n_neighbors', 
    'param_sampler', 
    'param_lda', 
    'mean_test_acc', 
    'mean_test_balanced_acc', 
    'mean_test_f1_weighted',
    'mean_fit_time'
]

# Sort by Balanced Accuracy (descending)
best_results = df_results[cols].sort_values(by='mean_test_balanced_acc', ascending=False)

# Get the best parameters
print("Best Parameters:", df_results.loc[best_results.index[0], 'params'])

print(best_results)

Best Parameters: {'knn__n_neighbors': 7, 'lda': LinearDiscriminantAnalysis(), 'sampler': 'passthrough'}
    param_knn__n_neighbors                          param_sampler  \
15                       7                            passthrough   
9                        5                            passthrough   
17                       7               SMOTE(random_state=1337)   
16                       7  RandomUnderSampler(random_state=1337)   
11                       5               SMOTE(random_state=1337)   
3                        3                            passthrough   
10                       5  RandomUnderSampler(random_state=1337)   
5                        3               SMOTE(random_state=1337)   
4                        3  RandomUnderSampler(random_state=1337)   
13                       7  RandomUnderSampler(random_state=1337)   
14                       7               SMOTE(random_state=1337)   
8                        5               SMOTE(random_state=1337)   